# Data Cleaning

In [1]:
# required libraries 
import pandas as pd 
import os 
from pathlib import Path
# Load the required libraries 
from rdflib import Graph, Literal, RDF, URIRef, Namespace
# rdflib knows about some namespaces, like FOAF 
from rdflib.namespace import FOAF, XSD 
# CHECK DATE 
import datetime

In [2]:
# parameters and URLs 
path = str(Path(os.path.abspath(os.getcwd())).parent.absolute())

batting_data = path +'/Homework_final_data/Batting_data.csv'
bowling_data = path +'/Homework_final_data/Bowling_data.csv'
match_detail = path +'/Homework_final_data/match_detail.csv'
players=  path +'/Homework_final_data/Players.csv'


# saving folder 
savePath = path + '/Homework_final_data/turtle/'

In [3]:
SO = Namespace("http://www.dei.unipd.it/database2/T20WC#")

# Matches

In [4]:
# Load the CSV files in memory 
matches_d = pd.read_csv(match_detail, sep=',')

In [5]:
#create the graph 
g = Graph() 

# Bind the namespaces to a prefix for more readable output 

g.bind("t20", SO) 

In [8]:
%%time 
#measure execution time 

#iterate over the league dataframe 
for index, row in matches_d.iterrows():
# Create the node to add to the Graph 
# the node has the namespace + the league id as URI 
    idU = "Match"+str(index) 
    matches = URIRef(SO[idU]) 
    # create the RDF node 
    #batting_team = URIRef(SO[row['Match_Id']]) 
    #bowling_team = URIRef(SO[row['Match_Id']])
 
  
# Add triples using store's add() method. 
    g.add((matches, RDF.type, SO.matches)) 
    # add the edge connecting the Movie and the Country
    g.add((matches, SO['score_card_bowling'], matches))
    g.add((matches, SO['score_card_batting'], matches))
    
    g.add((matches, SO['MatchId'], Literal(row['Match_Id'], datatype=XSD.int))) 
    g.add((matches, SO['Match_Date'], Literal(row['Match_Date'], datatype=XSD.string))) 
    g.add((matches, SO['Venue'], Literal(row['Venue'], datatype=XSD.string))) 
    g.add((matches, SO['Match_Conditions'], Literal(row['Match_Conditions'], datatype=XSD.string)))
    g.add((matches, SO['Player_of_Match'], Literal(row['Player_of_Match'], datatype=XSD.string))) 
    g.add((matches, SO['Toss'], Literal(row['Toss'], datatype=XSD.string))) 
    g.add((matches, SO['Result'], Literal(row['Result'], datatype=XSD.string)))
    
    
    


Wall time: 1.89 s


In [9]:
%%time 
# print all the data in the Turtle format 
print("--- saving serialization ---")
with open(savePath + 'match_detail.ttl', 'wb') as file: 
    file.write(g.serialize(format='turtle',encoding='utf-8'))

--- saving serialization ---
Wall time: 2.21 s


# BATTING TEAM


In [43]:
# Load the CSV files in memory 
batting_team = pd.read_csv(batting_data, sep=',',encoding= 'unicode_escape')

In [10]:
batting_team

,Player_id,Team,Opponent,Batsman,Wicket,R,BF,4s,6s,SR
Match_Id,,,,,,,,,,
1,2,Australia,New Zealand,Adam Gilchrist,c McMillan b Mills,1.0,3.0,0.0,0.0,33.33
1,1,Australia,New Zealand,Michael Clarke,c McMillan b Tuffey,7.0,4.0,0.0,1.0,175.00
1,11,Australia,New Zealand,Andrew Symonds,c ?ÿMcCullum b Mills,32.0,13.0,4.0,2.0,246.15
1,10,Australia,New Zealand,Ricky Ponting*,not out,98.0,55.0,8.0,5.0,178.18
1,9,Australia,New Zealand,Damien Martyn,b Mills,3.0,5.0,0.0,0.0,60.00
...,...,...,...,...,...,...,...,...,...,...
1850,107,England,Pakistan,Moeen Ali*,NaN,NaN,NaN,NaN,NaN,NaN
1850,242,England,Pakistan,Sam Curran,NaN,NaN,NaN,NaN,NaN,NaN
1850,235,England,Pakistan,David Willey,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# replacing na values in batting_data with not played
batting_team["Wicket"].fillna("Not played", inplace = True)
batting_team["R"].fillna("0", inplace = True)
batting_team["BF"].fillna("0", inplace = True)
batting_team["4s"].fillna("0", inplace = True)
batting_team["6s"].fillna("0", inplace = True)
batting_team["SR"].fillna("0", inplace = True)

In [15]:
batting_team

,Player_id,Team,Opponent,Batsman,Wicket,R,BF,4s,6s,SR
Match_Id,,,,,,,,,,
1,2,Australia,New Zealand,Adam Gilchrist,c McMillan b Mills,1,3,0,0,33.33
1,1,Australia,New Zealand,Michael Clarke,c McMillan b Tuffey,7,4,0,1,175
1,11,Australia,New Zealand,Andrew Symonds,c ?ÿMcCullum b Mills,32,13,4,2,246.15
1,10,Australia,New Zealand,Ricky Ponting*,not out,98,55,8,5,178.18
1,9,Australia,New Zealand,Damien Martyn,b Mills,3,5,0,0,60
...,...,...,...,...,...,...,...,...,...,...
1850,107,England,Pakistan,Moeen Ali*,Not played,0,0,0,0,0
1850,242,England,Pakistan,Sam Curran,Not played,0,0,0,0,0
1850,235,England,Pakistan,David Willey,Not played,0,0,0,0,0


In [28]:
batting_team.to_csv("Batting_data.csv",index="False")

In [44]:
#create the graph 
g = Graph() 

# Bind the namespaces to a prefix for more readable output 

g.bind("t20", SO) 

In [47]:
%%time 
#measure execution time 

#iterate over the league dataframe 
for index, row in batting_team.iterrows():
# Create the node to add to the Graph 
# the node has the namespace + the league id as URI 
    #batting = URIRef(SO["Match"+str(row['Match_Id'])])
    miD = "Match"+str(index) 
    batting = URIRef(SO[miD]) 

    
    piD = "Player"+str(row['Player_id']) 
    playerid = URIRef(SO[piD]) 
    
    #piD = URIRef(SO[row['Player_id']]) 
    #miD = URIRef(SO[row['Match_Id']]))
    #if (playerid == batting):
        
    # create the RDF node 
    
# Add triples using store's add() method. 
    g.add((batting, RDF.type, SO.Batting)) 
    
      # add the edge connecting the Movie and the Country
    g.add((URIRef(SO[row['Match_Id']]), SO['score_card_batting'], URIRef(SO[row['Match_Id']]))) 
    g.add((playerid, SO['batting_in'], batting))
    
    g.add((batting, SO['Player_id'], Literal(row['Player_id'], datatype=XSD.int)))
    g.add((batting, SO['Match_Id'], Literal(row['Match_Id'], datatype=XSD.int)))
    g.add((batting, SO['Team'], Literal(row['Team'], datatype=XSD.string)))
    g.add((batting, SO['Opponent'], Literal(row['Opponent'], datatype=XSD.string)))
    g.add((batting, SO['Batsman'], Literal(row['Batsman'], datatype=XSD.string))) 
    g.add((batting, SO['Wicket'], Literal(row['Wicket'], datatype=XSD.string))) 
    g.add((batting, SO['R'], Literal(row['R'], datatype=XSD.int)))
    g.add((batting, SO['BF'], Literal(row['BF'], datatype=XSD.int))) 
    g.add((batting, SO['4s'], Literal(row['4s'], datatype=XSD.int))) 
    g.add((batting, SO['6s'], Literal(row['6s'], datatype=XSD.int)))
    g.add((batting, SO['SR'], Literal(row['SR'], datatype=XSD.float)))
    
    
    

Wall time: 37.6 s


In [48]:
%%time 
# print all the data in the Turtle format 
print("--- saving serialization ---")
with open(savePath + 'batting.ttl', 'wb') as file: 
    file.write(g.serialize(format='turtle',encoding='utf-8'))

--- saving serialization ---
Wall time: 42.6 s


# Bowling Data

In [4]:
# Load the CSV files in memory 
bowling_data_in = pd.read_csv(bowling_data, sep=',',encoding= 'unicode_escape')

In [5]:
#create the graph 
g = Graph() 

# Bind the namespaces to a prefix for more readable output 

g.bind("t20", SO) 

In [6]:
%%time 
#measure execution time 

#iterate over the league dataframe 
for index, row in bowling_data_in.iterrows():
# Create the node to add to the Graph 
# the node has the namespace + the league id as URI 
    idU = "Match"+str(index) 
    bowling = URIRef(SO[idU]) 
    # create the RDF node 
    #piD = URIRef(SO[row['Player_id']]) 
    #miD = URIRef(SO[row['Match_Id']]) 
    piD = "Player"+str(row['Player_id']) 
    playerid = URIRef(SO[piD]) 
# Add triples using store's add() method. 
    g.add((bowling, RDF.type, SO.Bowling)) 
    # add the edge connecting the Movie and the Country
    g.add((URIRef(SO[row['Match_Id']]) , SO['score_card_bowling'], URIRef(SO[row['Match_Id']]))) 
    g.add((playerid , SO['bowling_in'], bowling))
    
    g.add((bowling, SO['Player_id'], Literal(row['Player_id'], datatype=XSD.int)))
    g.add((bowling, SO['Match_Id'], Literal(row['Match_Id'], datatype=XSD.int)))
    g.add((bowling, SO['bowlingTeam'], Literal(row['Team'], datatype=XSD.string)))
    g.add((bowling, SO['bowlingOpponent'], Literal(row['Opponent'], datatype=XSD.string)))
    g.add((bowling, SO['Bowler'], Literal(row['Bowler'], datatype=XSD.string))) 
    g.add((bowling, SO['Overs'], Literal(row['Overs'], datatype=XSD.float))) 
    g.add((bowling, SO['Maidens'], Literal(row['Maidens'], datatype=XSD.int)))
    g.add((bowling, SO['Runs'], Literal(row['Runs'], datatype=XSD.int))) 
    g.add((bowling, SO['Wickets'], Literal(row['Wickets'], datatype=XSD.int))) 
    g.add((bowling, SO['ER'], Literal(row['ER'], datatype=XSD.float)))

    

Wall time: 8.24 s


In [7]:
%%time 
# print all the data in the Turtle format 
print("--- saving serialization ---")
with open(savePath + 'bowling.ttl', 'wb') as file: 
    file.write(g.serialize(format='turtle',encoding='utf-8'))

--- saving serialization ---
Wall time: 9.06 s


# Players

In [34]:
# Load the CSV files in memory 
players_data = pd.read_csv(players, sep=',',encoding= 'unicode_escape')

In [35]:
#create the graph 
g = Graph() 

# Bind the namespaces to a prefix for more readable output 

g.bind("t20", SO) 

In [38]:
%%time 
#measure execution time 

#iterate over the league dataframe 
for index, row in players_data.iterrows():
#if index==0:continue
# Create the node to add to the Graph 
# the node has the namespace + the league id as URI 
    idU = "Player"+str(index) 
    play = URIRef(SO[idU])
    # create the RDF node 
    #piD = "player"+str(row['Player_id']) 
    #playerid = URIRef(SO[piD]) 
    # Add triples using store's add() method. 
    g.add((play, RDF.type, SO.matches)) 
        # add the edge connecting the Movie and the Country
        #g.add((miD, SO['score_card_batting'], batting)) 
        #g.add((playerid, SO['batting_in'],playerid))
        #g.add((playerid, SO['bowling_in'], playerid))
        
    g.add((play, SO['playerid'], Literal(row['Player_id'], datatype=XSD.int))) 
    g.add((play, SO['teamName'], Literal(row['TeamName'], datatype=XSD.string))) 
    g.add((play, SO['playername'], Literal(row['PlayerName'], datatype=XSD.string))) 
    g.add((play, SO['debutMatch'], Literal(row[' DebutMatch'], datatype=XSD.string))) 
    g.add((play, SO['seriesYear'], Literal(row[' SeriesYear'], datatype=XSD.string)))

    

Wall time: 278 ms


In [39]:
%%time 
# print all the data in the Turtle format 
print("--- saving serialization ---")
with open(savePath + 'Players.ttl', 'wb') as file: 
    file.write(g.serialize(format='turtle',encoding='utf-8'))


--- saving serialization ---
Wall time: 312 ms
